In [41]:
import io
import pandas as pd
from pprint import pprint
import requests

def ades_GetPointEau(numéro_département=None, nb_réseau=None):
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEau'
    query_params = {
        'type': '1',
        'filter': f'departement={numéro_département}&nbReseau={nb_réseau}',
        'rows': '500',
        'start': '0',
        'statut_qualif': '0'
    }
    response_GetPointEau = requests.post(url=url, data=query_params)
    return response_GetPointEau.json()['pointEaux']
    

def ades_GetPointEauIPSDataCSV(numéro_département=None, nombre_points=None):
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV'
    query_params = {
        'rows': -1,
        'start': 0,
        'filters': f'departement={numéro_département}',
        'nombre_points': nombre_points,
        'typeRecherche': 1,
        'date_debut': '',
        'date_fin': '',
        'moisAnnee': ''
    }
    response_GetPointEauIPSDataCSV = requests.get(url=url, params=query_params)
    return response_GetPointEauIPSDataCSV.content
    
    
numéro_département = 66
point_eau_liste = ades_GetPointEau(numéro_département, nb_réseau=0)
point_eau_liste_complétée = []
for i, p in enumerate(point_eau_liste):
    dic = {}
    dic['coordinates0'] = p['Coordinates'][0]
    dic['coordinates1'] = p['Coordinates'][1]
    dic['coord_com'] = p['Properties']['coord_com']
    dic['coord_ref0'] = p['Properties']['coord_ref'][0]
    dic['coord_ref1'] = p['Properties']['coord_ref'][1]
    dic['epsg_ref'] = p['Properties']['epsg_ref']
    dic['exist_mesures'] = p['Properties']['exist_mesures']
    if ' ' in p['Properties']['label']:
        dic['Identifiant BSS'] = p['Properties']['label'].split(' ')[0]
        dic['Ancien code national BSS'] = p['Properties']['label'].split(' ')[1].replace('(', '').replace(')', '')
    else:
        dic['Identifiant BSS'] = p['Properties']['label']
        dic['Ancien code national BSS'] = 'N/D'
    dic['position_captage'] = p['Properties']['position_captage']
    dic['type_pt_eau'] = p['Properties']['type_pt_eau']
    point_eau_liste_complétée.append(dic)
del point_eau_liste
df_point_eau = pd.DataFrame(point_eau_liste_complétée)
point_eau_ips_data_csv_bytes = ades_GetPointEauIPSDataCSV(numéro_département, nombre_points=len(point_eau_liste_complétée))
df_point_eau_ips_data = pd.read_csv(io.BytesIO(point_eau_ips_data_csv_bytes), sep=';')
df_point_eau = pd.merge(left=df_point_eau, right=df_point_eau_ips_data, on=['Identifiant BSS', 'Ancien code national BSS'])
del df_point_eau_ips_data
df_point_eau[df_point_eau['Ancien code national BSS'] == '10795X0001/S']

,coordinates0,coordinates1,coord_com,coord_ref0,coord_ref1,epsg_ref,exist_mesures,Identifiant BSS,Ancien code national BSS,position_captage,type_pt_eau,Type de suivi,Commune,Département,Nb de mesures,Date de début,Date de fin,Nb de réseaux
43,2.95833798658368,42.8589995016557,N,696591.5316611349,6195505.5129953185,2154,None,BSS002MHQQ,10795X0001/S,None,3,Piézomètre et qualitomètre,Salses-le-Château (66190),Pyrénées-Orientales (66),3034,27/02/2002,11/01/2011,9


In [ ]:
# curl 'https://ades.eaufrance.fr/Fiche/MesureGraph?CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couple=3&Lenght=3034'

# from urllib.parse import quote

def ades_FicheMesureGraph(row):
    url = 'https://ades.eaufrance.fr/Fiche/MesureGraph'
    query_params = {
        'CodeBSS': row['Ancien code national BSS'],
        'DateDebut': row['Date de début'],
        'DateFin': row['Date de fin'],
        'Couple': row['type_pt_eau'],
        'Length': row['Nb de mesures']
    }
    print(query_params)
    response_FicheMesureGraph = requests.get(url=url, params=query_params)
    return response_FicheMesureGraph.content

pprint(
    ades_FicheMesureGraph(df_point_eau.loc[43,]).decode()
)


In [65]:
# curl -D - 'https://ades.eaufrance.fr/Fiche/MesureTabDonnees' -X POST \
# --data-raw 'draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&start=0&length=2000&search%5Bvalue%5D=&search%5Bregex%5D=false&CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couples=3&KnowNbResult=false' -o /dev/null

from urllib.parse import unquote

s = 'draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&start=0&length=2000&search%5Bvalue%5D=&search%5Bregex%5D=false&CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couples=3&KnowNbResult=false'
pprint(
    unquote(s).split('&')
)

['draw=1',
 'columns[0][data]=0',
 'columns[0][name]=',
 'columns[0][searchable]=true',
 'columns[0][orderable]=true',
 'columns[0][search][value]=',
 'columns[0][search][regex]=false',
 'columns[1][data]=1',
 'columns[1][name]=',
 'columns[1][searchable]=true',
 'columns[1][orderable]=true',
 'columns[1][search][value]=',
 'columns[1][search][regex]=false',
 'columns[2][data]=2',
 'columns[2][name]=',
 'columns[2][searchable]=true',
 'columns[2][orderable]=true',
 'columns[2][search][value]=',
 'columns[2][search][regex]=false',
 'columns[3][data]=3',
 'columns[3][name]=',
 'columns[3][searchable]=true',
 'columns[3][orderable]=true',
 'columns[3][search][value]=',
 'columns[3][search][regex]=false',
 'columns[4][data]=4',
 'columns[4][name]=',
 'columns[4][searchable]=true',
 'columns[4][orderable]=true',
 'columns[4][search][value]=',
 'columns[4][search][regex]=false',
 'columns[5][data]=5',
 'columns[5][name]=',
 'columns[5][searchable]=true',
 'columns[5][orderable]=true',
 'colu